In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adityasd314","key":"7e0556eb21f4b18f3c9df0e8d5c63d5e"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                          title                                                      size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------  --------------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
jayaantanaath/student-habits-vs-academic-performance         Student Habits vs Academic Performance                    19512  2025-04-12 10:49:08.663000          28166        499  1.0              
adilshamim8/social-media-addiction-vs-relationships          Students' Social Media Addiction                           7851  2025-05-10 14:38:02.713000           4217         63  1.0              
orvile/health-and-sleep-relation-2024                        Health and Sleep relation                                  2666  2025-05-08 12:35:19.687000           1745         33  1.0              
omniamahmo

In [ ]:
!kaggle datasets download -d wcukierski/enron-email-dataset

Dataset URL: https://www.kaggle.com/datasets/wcukierski/enron-email-dataset
License(s): copyright-authors
 98% 350M/358M [00:02<00:00, 186MB/s]
100% 358M/358M [00:02<00:00, 161MB/s]


In [ ]:
import zipfile
with zipfile.ZipFile("enron-email-dataset.zip", "r") as zip_ref:
  zip_ref.extractall(".")

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import multiprocessing
import seaborn as sns
import email
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("./emails.csv")

### Extract headers

In [ ]:
# now we add those fields into our 'df' dataframe
def get_field(field, messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(e.get(field))
    return column

In [ ]:
df['date'] = get_field("Date", df['message'])
df['subject'] = get_field("Subject", df['message'])
df['X-From'] = get_field("X-From", df['message'])
df['X-To'] = get_field("X-To", df['message'])
df.head(3)

,file,message,date,subject,X-From,X-To
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,Phillip K Allen,Leah Van Arsdall


### Extract Message Body

In [ ]:
def body(messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(e.get_payload())
    return column

df['body'] = body(df['message'])
df.head(3)

,file,message,date,subject,X-From,X-To,body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,Phillip K Allen,Leah Van Arsdall,test successful. way to go!!!


## Data Cleaning and Transformation

In [ ]:
import datetime
from dateutil import parser

# We are not using this field for now, just in case if needed later 
# this is sample example
x = parser.parse("Fri, 4 May 2001 13:51:00 -0700 (PDT)")
print(x.strftime("%d-%m-%Y %H:%M:%S"))

04-05-2001 13:51:00


In [ ]:
def change_type(dates):
    column = []
    for date in dates:
        column.append(parser.parse(date).strftime("%d-%m-%Y %H:%M:%S"))
    return column

df['date'] = change_type(df['date'])
df.head(2)

,file,message,date,subject,X-From,X-To,body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,14-05-2001 16:39:00,,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,04-05-2001 13:51:00,Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...


In [ ]:
def replace_empty_with_nan(subject):
    column = []
    for val in subject:
        if (val == ""):
            column.append(np.nan)
        else:
            column.append(val)
    return column

In [ ]:
df['subject'] = replace_empty_with_nan(df['subject'])
df['X-To'] = replace_empty_with_nan(df['X-To'])
df['X-From'] = replace_empty_with_nan(df['X-From'])

In [ ]:
df.isnull().sum()



,0
file,0
message,0
date,0
subject,19187
X-From,29
X-To,9153
body,0


In [ ]:
# calculate percentage of missing values
miss = df.isnull().sum()
miss = miss[miss>0]
miss = miss / df.shape[0]
miss

,0
subject,0.037083
X-From,0.000056
X-To,0.017690


In [ ]:
# No significant loss of information
df.dropna(axis=0, inplace=True)


In [ ]:
df.isnull().sum(), df.shape


(file       0
 message    0
 date       0
 subject    0
 X-From     0
 X-To       0
 body       0
 dtype: int64,
 (489236, 7))

In [ ]:

df.head(3)


,file,message,date,subject,X-From,X-To,body
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,04-05-2001 13:51:00,Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,18-10-2000 03:00:00,Re: test,Phillip K Allen,Leah Van Arsdall,test successful. way to go!!!
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,31-08-2000 05:07:00,Re: Hello,Phillip K Allen,Greg Piper,Let's shoot for Tuesday at 11:45.


In [ ]:
cols_to_drop = ['file','message']

In [ ]:
df.drop(cols_to_drop, axis=1, inplace=True)
df.head(3)

,date,subject,X-From,X-To,body
1,04-05-2001 13:51:00,Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...
2,18-10-2000 03:00:00,Re: test,Phillip K Allen,Leah Van Arsdall,test successful. way to go!!!
4,31-08-2000 05:07:00,Re: Hello,Phillip K Allen,Greg Piper,Let's shoot for Tuesday at 11:45.


In [ ]:
# save the file
df.to_csv("cleaned_data.csv", index=False)